<a href="https://colab.research.google.com/github/amanotk/python-resume-public/blob/master/report/GaleShapley.ipynb" target="_blank">
<img src="https://colab.research.google.com/assets/colab-badge.svg" style="vertical-align: text-top;">
</a>

# 課題4. 進学選択問題

Gale-Shapleyアルゴリズムを使って駒場進学選択問題を解いてみよう．ここで，

- 学科数と学生数は異なる．
- 各学科は学科ごとに決められた定員までは受け入れが可能である．
- 学生は必ずしも全ての学科を志望する必要は無い．
- 各学科は各科目の試験成績の重みを付け平均点をもとに学生のランキングを決定する．ただし重み係数は各学科ごとに異なる．
- 同点となる場合については考えなくてよい．

を条件とする．  
このとき，学科数，科目数，学科定員，各学科ごとの重み係数，各学生の試験成績および志望学科リストのデータから，実際に学生を各学科に配属せよ．

各学科および学生のデータは

- テスト用（学生数40名）: https://amanotk.github.io/python-resume-public/report/data/gsmatch1.json
- 本番用（学生数310名）: https://amanotk.github.io/python-resume-public/report/data/gsmatch2.json

を用いること．データは以下のようなJSON形式になっているので簡単に読み込むことができる．


```python
{
    # 学科の情報
    "department": {
        # 学科数
        "numdept": 10,
        # 学科ID
        "deptid" : [ 1, 2, 3, 4, 5, 6, 7, 8, 9, 10 ],
        # 学科名
        "deptname": [
            "数学科",
            "情報科学科",
            "物理学科",
            "天文学科",
            "地球惑星物理学科",
            "地球惑星環境学科",
            "化学科",
            "生物化学科",
            "生物学科",
            "生物情報学科"
        ],
        # 学科定員
        "quota": [ 45, 28, 70, 9, 32, 20, 45, 20, 10, 20 ],
        # 科目数
        "numsubj": 5,
        # 各学科の各科目の重み
        "weight": [
            # 学科1の各科目の重み
            [ 0.6, 0.1, 0.1, 0.1, 0.1 ],
            # 以下省略（学科数分だけ続く）
        ]
    },
    # 学生の情報
    "student": {
        # 学生数
        "numstd": 310,
        # 学生ID
        "stdid" : [
            1,
            # 以下省略（学生数分だけ続く）
        ],
        # 学生の名前
        "stdname": [
            "Name0001",
            # 以下省略（学生数分だけ続く）
        ],
        # 学生の点数
        "score": [
            # 学生1の各科目の点数
            [ 61.3, 64.8, 76.8, 49.9, 56.3 ],
            # 以下省略（学生数分だけ続く）
        ],
        # 学生の希望学科リストを志望順で（負の値はそれ以上希望が無いことを意味する）
        "preference": [
            [ 5, 3, 4, -1, -1, -1, -1, -1, -1, -1 ],
            # 以下省略（学生数分だけ続く）
        ]
    }
}
```

なお，学生の志望リストは学科IDで表し，志望学科リストに負のIDが指定されている場合は，それ以上志望学科が無いことを表すものとする．

アルゴリズムを適用した結果は

- https://amanotk.github.io/python-resume-public/report/data/gsmatch1_result.json
- https://amanotk.github.io/python-resume-public/report/data/gsmatch2_result.json

に置いてあるので各自で適宜確認すること．

In [ ]:
from IPython.display import display
import pandas as pd
import urllib.request
import json

# ファイルの場所
baseurl = 'https://amanotk.github.io/python-resume-public/report/data/'
infile1  = baseurl + 'gsmatch1.json'
infile2  = baseurl + 'gsmatch2.json'
outfile1 = baseurl + 'gsmatch1_result.json'
outfile2 = baseurl + 'gsmatch2_result.json'

In [ ]:
def check_result(obj, outfile):
    "配属結果のjsonオブジェクトobjとoutfileで指定された正解を比較する"
    with urllib.request.urlopen(outfile) as response:
        json_obj = json.loads(response.read().decode("utf-8"))
    for key in obj:
        if obj[key] != json_obj[key]:
            print("Error: ", key, obj[key], json_obj[key])
    print("Success!")

# 使い方の例（正解同士で比較）
with urllib.request.urlopen(outfile1) as response:
    test_obj = json.loads(response.read().decode("utf-8"))
check_result(test_obj, outfile1)

In [ ]:
# データ読み込み例
infile = infile1

with urllib.request.urlopen(infile) as response:
   json_obj = json.loads(response.read().decode("utf-8"))

## 学科情報表示

In [ ]:
department = json_obj["department"]
numsubj = department["numsubj"]
numdept = department["numdept"]

# 学科
df1 = pd.DataFrame(
    data={"学科名": department["deptname"], "定員": department["quota"]}, index=department["deptid"]
)
df2 = pd.DataFrame(
    data=department["weight"],
    index=department["deptid"],
    columns=["重み{:1d}".format(i + 1) for i in range(numsubj)],
)
df_department = pd.merge(df1, df2, left_index=True, right_index=True)

# 表示
df_department

## 学生情報表示

In [ ]:
student = json_obj["student"]

# 学生
df3 = pd.DataFrame(
    data=student["score"],
    index=student["stdid"],
    columns=["得点{:1d}".format(i + 1) for i in range(numsubj)],
)
df4 = pd.DataFrame(
    data=student["preference"],
    index=student["stdid"],
    columns=["第{:1d}希望".format(i + 1) for i in range(numdept)],
)
df_student = pd.merge(df3, df4, left_index=True, right_index=True)

# 表示
df_student

## 学生配属